In [1]:
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pandas as pd
import datetime
#with open('Ethan.json') as json_file:
 #   data = json.load(json_file)
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
import pydotplus
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder


In [2]:
#Commence cleaning of data

#big combo can easily be changed for various levels of interest
fwjr = pd.read_parquet("fwjr_2018_1110_1115.parquet")
classads = pd.read_parquet("classAds_2018_1110_1114.parquet")
fwjr['CRAB_Id'] = fwjr['fwjr_id'].str.extract(r'([0-9]+)-[0-9]+') 
fwjr['CRAB_Retry'] = fwjr['fwjr_id'].str.extract(r'[0-9]+-([0-9])+') 
fwjr['Big_Combo'] = fwjr.task+fwjr.fwjr_id
classads['Big_Combo'] = classads.CRAB_Workflow+classads['CRAB_Id']+'-'+classads['CRAB_Retry']

In [3]:

#cleanads and fwjr have dropped the duplicates in Big_Combo
print(len(classads))
print(len(classads.drop_duplicates(subset = 'Big_Combo')))
cleanads = classads.drop_duplicates(subset = 'Big_Combo')
#cleanads = classads[classads.CRAB_Retry == '0']
print(len(cleanads))

print(len(fwjr))
print(len(fwjr.drop_duplicates(subset = 'Big_Combo')))
fwjr = fwjr.drop_duplicates(subset = 'Big_Combo')

1773993
1773993
1773993
1183693
1180174


In [4]:
#cleanads = cleanads[cleanads['CMS_TaskType'].notna()]
#for i in cleanads:
    #cleanads[i] = pd.to_numeric(cleanads[i],errors='ignore',downcast = 'float')

In [5]:

#creates the isboth category which is the target variable

cleanads['isboth'] = np.where(cleanads['Big_Combo'].isin(fwjr['Big_Combo']), 1, 0)
#cleanads = (cleanads.sort_values(by = ['isboth'],ascending = False)).reset_index()
print(len(cleanads[cleanads.isboth == 1]))
print(len(cleanads))
print(len(cleanads[cleanads.isboth == 1])/len(cleanads))
print(len(fwjr))


fwjr['isboth'] = np.where(fwjr['Big_Combo'].isin(classads['Big_Combo']), 1, 0)


1109407
1773993
0.625372817141894
1180174


In [6]:

#analyzes cleanads to make decision on dates for goodads
dates = []
shared = []
length = []
for i in cleanads.day.unique():
    dates.append(i)
    
dates.sort()
for i in dates:
    shared.append(sum(cleanads[cleanads.day == i]['isboth'])/len(cleanads[cleanads.day == i])*100%100)
    length.append(len(cleanads[cleanads.day == i]))
print(dates)    
print(shared)
print(length)

[1541808000.0, 1541894400.0, 1541980800.0, 1542067200.0, 1542153600.0]
[46.60795161582519, 64.64288745744169, 66.82185273159145, 70.19116912273017, 64.6189737581887]
[358795, 212062, 515725, 292202, 395209]


In [7]:

#analyzes fwjr to make decision on dates for goodfwjr
dates = []
shared = []
length = []
for i in fwjr.day.unique():
    dates.append(i)
    
dates.sort()
for i in dates:
    shared.append(sum(fwjr[fwjr.day == i]['isboth'])/len(fwjr[fwjr.day == i])*100%100)
    length.append(len(fwjr[fwjr.day == i]))
print(dates)    
print(shared)
print(length)

[1541548800, 1541635200, 1541721600, 1541808000, 1541894400, 1541980800, 1542067200, 1542153600, 1542240000]
[0.0, 0.0, 0.0, 96.37532254281022, 95.45128979294995, 99.07334372986675, 99.84109569957073, 98.1514763536887, 35.801575203252035]
[1, 1, 17, 106575, 181502, 332162, 229698, 251498, 78720]


In [8]:
#first analysis step not cleaning up

print('Number of Unique Values in Each Field of FWJR:\n')
print(fwjr[fwjr.day.isin([1541808000, 1541894400, 1541980800, 1542067200, 1542153600])].nunique())
print('')
print('Number of Unique Values in Each Field of Shared:\n')
print(cleanads[cleanads.isboth == 0][cleanads.day.isin([1541894400.0, 1541980800.0, 1542067200.0, 1542153600.0])].nunique())
print('')
print('Number of Unique Values in Each Field of Cleanads:\n')
print(cleanads[cleanads.day.isin([1541894400.0, 1541980800.0, 1542067200.0, 1542153600.0])].nunique())

Number of Unique Values in Each Field of FWJR:

day                   5
d_dataset_id       1570
task               4252
fwjr_id           52571
host                 14
files_read           82
CRAB_Id            9644
CRAB_Retry           10
Big_Combo       1101435
isboth                2
dtype: int64

Number of Unique Values in Each Field of Shared:



/Users/ethangreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


CMS_TaskType          0
RecordTime       205847
day                   4
d_dataset_id       1120
CRAB_Workflow      2590
CRAB_Id           13684
CRAB_Retry           53
CRAB_UserHN         167
Campaign            167
ErrorType             0
RequestCpus           3
ScheddName           14
WallClockHr       41001
Status                2
Big_Combo        473018
isboth                1
dtype: int64

Number of Unique Values in Each Field of Cleanads:

CMS_TaskType           0
RecordTime        318180
day                    4
d_dataset_id        1550
CRAB_Workflow       4353
CRAB_Id            15151
CRAB_Retry            53
CRAB_UserHN          188
Campaign             188
ErrorType              0
RequestCpus            3
ScheddName            15
WallClockHr        66165
Status                 2
Big_Combo        1415198
isboth                 2
dtype: int64


In [9]:
cleanads.head()

,CMS_TaskType,RecordTime,day,d_dataset_id,CRAB_Workflow,CRAB_Id,CRAB_Retry,CRAB_UserHN,Campaign,ErrorType,RequestCpus,ScheddName,WallClockHr,Status,Big_Combo,isboth
0,None,1542002390000,1.541981e+09,13304950,181022_135145:dpgilber_crab_miniaod_94x_ttsl_f...,93,4,dpgilber,crab_dpgilber,None,8,crab3@vocms0197.cern.ch,3.7063888888888887,Completed,181022_135145:dpgilber_crab_miniaod_94x_ttsl_f...,1
1,None,1541990466000,1.541981e+09,13293064,181022_140525:dpgilber_crab_miniaod_94x_ttsl_z...,218,6,dpgilber,crab_dpgilber,None,8,crab3@vocms0144.cern.ch,4.1322222222222225,Completed,181022_140525:dpgilber_crab_miniaod_94x_ttsl_z...,1
2,None,1542211835000,1.542154e+09,13293064,181022_140525:dpgilber_crab_miniaod_94x_ttsl_z...,77,7,dpgilber,crab_dpgilber,None,8,crab3@vocms0144.cern.ch,4.835277777777778,Completed,181022_140525:dpgilber_crab_miniaod_94x_ttsl_z...,1
3,None,1541967781000,1.541894e+09,13293064,181022_140525:dpgilber_crab_miniaod_94x_ttsl_z...,125,9,dpgilber,crab_dpgilber,None,8,crab3@vocms0144.cern.ch,5.868333333333333,Completed,181022_140525:dpgilber_crab_miniaod_94x_ttsl_z...,1
4,None,1541859310000,1.541808e+09,13306621,181022_143003:dpgilber_crab_miniaod_94x_wjets_...,16,3,dpgilber,crab_dpgilber,None,8,crab3@vocms0107.cern.ch,3.6013888888888888,Completed,181022_143003:dpgilber_crab_miniaod_94x_wjets_...,1


In [10]:
fwjr.head()

,day,d_dataset_id,task,fwjr_id,host,files_read,CRAB_Id,CRAB_Retry,Big_Combo,isboth
0,1542153600,13289820,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3,3-0,vocms0137.cern.ch,8,3,0,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_33-0,1
1,1542153600,13289820,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3,1-14-0,vocms0137.cern.ch,4,1,4,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3...,1
2,1542153600,13289820,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3,1-13-0,vocms0137.cern.ch,3,1,3,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3...,1
3,1542153600,13289820,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3,1-11-0,vocms0137.cern.ch,3,1,1,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3...,1
4,1542153600,13289820,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_3,4-0,vocms0137.cern.ch,9,4,0,181113_162805:mthiel_crab_QCD_Pt_800_Moriond_34-0,1


In [11]:
#cleaning again
#the Campaign column is just a copy of the CRAB_UserHN column but starting with 'crab_' 
cleanads = cleanads.drop(['Campaign','CMS_TaskType','ErrorType'],axis = 1)

In [12]:
print(len(cleanads))
print(sum(cleanads[cleanads.Status == 'Removed'].isboth))
#place the set of good days in the arrays. done at user discression 
goodfwjr = fwjr[fwjr.day.isin([1541808000, 1541894400, 1541980800, 1542067200, 1542153600])]
goodads = cleanads[cleanads.day.isin([1541894400.0, 1541980800.0, 1542067200.0, 1542153600.0])][cleanads.Status == 'Completed']

1773993
2


/Users/ethangreen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [13]:
#original percentage in both
print('Classads perspective:')
print(sum(goodads.isboth)/len(goodads))
print('FWJR perspective:')
print(sum(goodfwjr.isboth)/len(goodfwjr))

Classads perspective:
0.6657593699233956
FWJR perspective:
0.9816503016519359


In [14]:
for i in cleanads.ErrorType.unique().tolist():
    print(i)
    print(sum(goodads[goodads.ErrorType == i].isboth)/len(goodads[goodads.ErrorType == i]))
    print(len(goodads[goodads.ErrorType == i].isboth))

AttributeError: 'DataFrame' object has no attribute 'ErrorType'

In [ ]:
#last basic cleaning step
#Removes all errors
print('Classads false negative percentage removed:')
print(sum(goodads[goodads.ErrorType != 'Success'].isboth)/len(goodads[goodads.ErrorType == 'Success']))
goodads = goodads[goodads.ErrorType == 'Success']
print('Percent good with errors removed')
print(sum(goodads.isboth)/len(goodads))
#goodads = goodads[~goodads.CRAB_Workflow.isin(badworkflow)]

In [ ]:
print(sum(goodads[goodads.ScheddName == 'crab3@vocms0196.cern.ch'].isboth)/len(goodads[goodads.ScheddName == 'crab3@vocms0196.cern.ch']))
print(len(goodads[goodads.ScheddName == 'crab3@vocms0196.cern.ch']))

goodads = goodads[goodads.ScheddName != 'crab3@vocms0196.cern.ch']
print('Percent good with errors removed')
print(sum(goodads.isboth)/len(goodads))
print(len(goodads))

In [ ]:
#my algorithm starting with goodads

#creates an array for each field in the dataframe
#each value in the array represents a unique value in that field
#outputs the standard deviation then the mean of this array
#these numbers can be used to tell how useful each field should be for seperating based on isboth
#a useful field is one with a lower standard deviation and a higher mean
#Note: All values in the array are between 0 and 0.5 so the closer the second number is to 0.5,
#     the better that field should be as a predictor.

#iterates through fields in goodads
for field in goodads:
    #creates and refreshes an array for each field
    prayer = []
    weight = []
    #skips to the next field if the current one has too many unique values for the code to run quickly
    if len(goodads[field].unique())>5000:
        print(len(goodads[field].unique()))
        continue
    #iterates through unique values of the field we are looking at.
    for i in goodads[field].unique().tolist():
        #not sure why but sometimes there arent actually any jobs with a certain value that comes up
        #we have to skip these values to avoid division by zero in the next step
        if len(goodads[goodads[field] == i]['isboth']) == 0:
            continue
        #creates a value for the array equal to the absolute value of 0.5 - x
        #where x is the precentage of jobs in both with with one particular value in the current fields
        newval = abs(0.5-sum(goodads[goodads[field] == i]['isboth'])/len(goodads[goodads[field] == i]['isboth']))
        weight.append(len(goodads[field] == i))
        prayer.append(newval)
    #print the field name, standard deviation of usefulness array,  
    #mean of usefulness array, and number of unique values in the field
    print(field)
    print(np.std(prayer))
    #print(np.average(prayer,weights = weight))
    print(np.average(prayer))
    print(len(goodads[field].unique()))
    

In [ ]:


#Creates an array of bad workflow to be removed from the set

badworkflow = []
#iterates through unique values of the desired field, in this case 'workflow'
for i in goodads.CRAB_Workflow.unique().tolist():    
    #seperates good workflow from bad workflow by skipping any workflow for which more than 
    #half the jobs are in both
    if sum(goodads[goodads.CRAB_Workflow == i].isboth)/len(goodads[goodads.CRAB_Workflow == i].isboth)>= .3:
        continue
    #print(i)
    #print(len(goodads[goodads.CRAB_Workflow == i].isboth))
    #adds the badworkflow to the array
    badworkflow.append(i)
print(len(badworkflow))
print(len(goodads.CRAB_Workflow.unique().tolist()))

In [ ]:

#not enough jobs per id for this to be a valid method

#Creates an array of bad Id's to be removed from the set

badid = []
#iterates through unique values of the desired field, in this case 'task'
for i in goodads.CRAB_Id.unique().tolist():    
    continue
    #seperates good tasks from bad tasks by skipping any tasks for which more than 
    #half the jobs are in both
    if sum(goodads[goodads.CRAB_Id == i].isboth)/len(goodads[goodads.CRAB_Id == i].isboth)> 0.5:
        continue
    #print(i)
    #print(sum(goodads[goodads.CRAB_Id == i].isboth)/len(goodads[goodads.CRAB_Id == i].isboth))
    print(len(goodads[goodads.CRAB_Id == i].isboth))
    #adds the badtasks to the array
    badworkflow.append(i)


In [ ]:

#Retry is not useful, only one of them actually has less than like 80% good jobs
# and that retry holds a single job
#Creates an array of bad retry to be removed from the set

badretry = []
#iterates through unique values of the desired field, in this case 'task'
for i in goodads.CRAB_Retry.unique().tolist():
    continue
    #seperates good tasks from bad tasks by skipping any tasks for which more than 
    #half the jobs are in both
    if sum(goodads[goodads.CRAB_Retry == i].isboth)/len(goodads[goodads.CRAB_Retry == i].isboth)> 0.5:
        print(i)
        print(len(goodads[goodads.CRAB_Retry == i].isboth))
        print(sum(goodads[goodads.CRAB_Retry == i].isboth)/len(goodads[goodads.CRAB_Retry == i].isboth))
        continue
    print(i)
    print(len(goodads[goodads.CRAB_Retry == i].isboth))
    #adds the badtasks to the array
    badretry.append(i)


In [ ]:
len(goodads.CRAB_Workflow.unique())

In [ ]:
print('Previous percentage and length of jobs in both from classads on reasonable days perspctive:')
print(str(sum(goodads.isboth)/len(goodads.isboth))+'\t'+str(len(goodads.isboth)))

print('Percentage and length of only good workflow:')
print(str(sum(goodads[~goodads.CRAB_Workflow.isin(badworkflow)].isboth)/len(goodads[~goodads.CRAB_Workflow.isin(badworkflow)].isboth))+'\t'+str(len(goodads[~goodads.CRAB_Workflow.isin(badworkflow)].isboth)))

print('Percentage and length of bad workflow:')
print(str(sum(goodads[goodads.CRAB_Workflow.isin(badworkflow)].isboth)/len(goodads[goodads.CRAB_Workflow.isin(badworkflow)].isboth))+'\t'+str(len(goodads[goodads.CRAB_Workflow.isin(badworkflow)].isboth)))


In [ ]:
#prints ScheddName, percent of good, number of jobs in each schedd

for i in goodads.ScheddName.unique():
    print(i)
    print(sum(goodads[goodads.ScheddName == i].isboth)/len(goodads[goodads.ScheddName == i]))
    print(len(goodads[goodads.ScheddName == i]))

In [ ]:
goodads = goodads[~goodads.CRAB_Workflow.isin(badworkflow)]

In [ ]:
baddads = goodads[goodads.CRAB_Workflow.isin(badworkflow)]
for i in baddads.ScheddName.unique():
    print(i)
    print(sum(baddads[baddads.ScheddName == i].isboth)/len(baddads[baddads.ScheddName == i]))
    print(len(baddads[baddads.ScheddName == i]))

In [ ]:
print('Original percentage and length of jobs in both from classads on reasonable days perspctive:')
print(str(sum(goodads.isboth)/len(goodads.isboth))+'\t'+str(len(goodads.isboth)))
print('Percentage and length of only Success:')
print(str(sum(goodads[goodads.ErrorType == 'Success'].isboth)/len(goodads[goodads.ErrorType == 'Success'].isboth))+'\t'+str(len(goodads[goodads.ErrorType == 'Success'].isboth)))

print('Percentage and length of only Completed:')
print(str(sum(goodads[goodads.Status == 'Completed'].isboth)/len(goodads[goodads.Status == 'Completed'].isboth))+'\t'+str(len(goodads[goodads.Status == 'Completed'].isboth)))

print('Percentage and length of not Success:')
print(str(sum(goodads[goodads.ErrorType != 'Success'].isboth)/len(goodads[goodads.ErrorType != 'Success'].isboth))+'\t'+str(len(goodads[goodads.ErrorType != 'Success'].isboth)))

print('Percentage and length of not Completed:')
print(str(sum(goodads[goodads.Status != 'Completed'].isboth)/len(goodads[goodads.Status != 'Completed'].isboth))+'\t'+str(len(goodads[goodads.Status != 'Completed'].isboth)))

print('Percentage and length of only Completed Success:')
print(str(sum(goodads[goodads.Status == 'Completed'][goodads.ErrorType == 'Success'].isboth)/len(goodads[goodads.Status == 'Completed'][goodads.ErrorType == 'Success'].isboth))+'\t'+str(len(goodads[goodads.Status == 'Completed'][goodads.ErrorType == 'Success'].isboth)))
print('Percentage and length of not successes and removed:')
print(str(sum(goodads[(goodads.ErrorType != 'Success') | (goodads.Status == 'Removed')].isboth)/len(goodads[(goodads.ErrorType != 'Success') | (goodads.Status == 'Removed')].isboth))+'\t'+str(len(goodads[(goodads.ErrorType != 'Success') | (goodads.Status == 'Removed')].isboth)))

In [ ]:
goodads = cleanads[cleanads.day.isin([1551484800, 1551571200, 1551657600, 1551744000])]
goodads = goodads[goodads.Status == 'Completed'][goodads.ErrorType == 'Success']

In [ ]:
#Uniquely good/bad method

#prints the field
#number of unique values only in the good/bad set
#total number of uunique values
#number of jobs with values in the only good/bad set

badarray = []
gooddict = {}
for i in goodads:
    if len(goodads[i].unique())/len(goodads)>0.01:
        print(i)
        continue
    print(i)
    badarray = [x for x in goodads[i][goodads.isboth == 1].unique().tolist() if x not in goodads[i][goodads.isboth == 0].unique().tolist()]
    print(len(badarray))
    print(len(goodads[i].unique()))
    gooddict.update({i:badarray})
    print(len(goodads[goodads[i].isin(badarray)]))

In [ ]:
#Uniquely good/bad method

#prints the field
#number of unique values only in the good/bad set
#total number of uunique values
#number of jobs with values in the only good/bad set

badarray = []
baddict = {}
for i in goodads:
    if len(goodads[i].unique())/len(goodads)>0.01:
        print(i)
        continue
    print(i)
    badarray = [x for x in goodads[i][goodads.isboth == 0].unique().tolist() if x not in goodads[i][goodads.isboth == 1].unique().tolist()]
    print(len(badarray))
    print(len(goodads[i].unique()))
    baddict.update({i:badarray})
    print(len(goodads[goodads[i].isin(badarray)]))

In [ ]:
sum(goodads.isboth)/len(goodads)

In [ ]:
1-sum(goodads[goodads.ErrorType != 'Success'].isboth)/len(goodads[goodads.ErrorType != 'Success'])

In [ ]:
sum(goodads[~goodads.CRAB_Workflow.isin(baddict['CRAB_Workflow'])].isboth)/len(goodads[~goodads.CRAB_Workflow.isin(baddict['CRAB_Workflow'])].isboth)

In [ ]:
goodads = goodads[~goodads.CRAB_Workflow.isin(baddict['CRAB_Workflow'])]

In [ ]:
goodfwjr = fwjr[fwjr.day.isin([1551484800, 1551571200, 1551657600, 1551744000])]

In [15]:
#Uniquely good/bad method

#prints the field
#number of unique values only in the good/bad set
#total number of uunique values
#number of jobs with values in the only good/bad set

badarray = []
baddict = {}
for i in goodfwjr:
    if len(goodfwjr[i].unique())/len(goodfwjr)>0.01:
        print(i)
        continue
    print(i)
    badarray = [x for x in goodfwjr[i][goodfwjr.isboth == 0].unique().tolist() if x not in goodfwjr[i][goodfwjr.isboth == 1].unique().tolist()]
    print(len(badarray))
    print(len(goodfwjr[i].unique()))
    baddict.update({i:badarray})
    print(len(goodfwjr[goodfwjr[i].isin(badarray)]))

day
0
5
0
d_dataset_id
14
1570
16810
task
61
4252
19521
fwjr_id
host
0
14
0
files_read
0
82
0
CRAB_Id
0
9644
0
CRAB_Retry
0
10
0
Big_Combo
isboth
1
2
20211


In [ ]:
len(goodfwjr)

In [16]:
baddict['task']

['181114_163242:szhaozho_crab_PromptRECODs0',
 '181113_101754:szhaozho_crab_PromptRECOMB4',
 '181113_111235:szhaozho_crab_PromptRECOMB11',
 '181110_183608:gkaratha_crab_FastSim_GenBToKJpsiToEE_NonD_muFilter_try2',
 '181110_175657:gkaratha_crab_FastSim_GenBToKEE_NonD_muFilter_try2',
 '181113_104931:szhaozho_crab_PromptRECOMB7',
 '181114_132427:mlotti_crab_Hplus2hw_4l_HLT_MC_m350_2',
 '181114_101942:mlotti_crab_Hplus2hw_4l_HLT_MC_m350_1',
 '181114_004704:katatar_crab_skim_HIHardProbes_RAW_run326392_userInputFiles',
 '181112_020816:atittert_crab_nmssmSignalCascadeV01_13TeV2017_processMc02_94X_titterton_P1_2000sq_R0p555',
 '181112_020958:atittert_crab_nmssmSignalCascadeV01_13TeV2017_processMc02_94X_titterton_P3_2100sq_R0p555',
 '181112_020306:atittert_crab_nmssmSignalCascadeV01_13TeV2017_processMc02_94X_titterton_P3_1900sq_R0p99',
 '181112_020725:atittert_crab_nmssmSignalCascadeV01_13TeV2017_processMc02_94X_titterton_P1_1800sq_R0p555',
 '181112_020909:atittert_crab_nmssmSignalCascadeV01_13

In [17]:
prebadtasks = ['190228_210829:algomez_crab_tthbb13_PostProcMEAnalysis_noME_SingleMuon_Run2018A_102X_v00',
 '190304_132159:andrea_crab_Zprime_FV_500_gb1dbs0_2_step2_80XRM',
 '190304_141655:andrea_crab_Zprime_FV_500_gb1dbs0_6_step2_80XRM',
 '190304_175231:andrea_crab_Zprime_FV_500_gb1dbs0_9_step2_80XRM',
 '190304_181225:andrea_crab_Zprime_FV_500_gb1dbs0_12_step2_80XRM',
 '190304_180041:andrea_crab_Zprime_FV_500_gb1dbs0_10_step2_80XRM',
 '190304_134855:andrea_crab_Zprime_FV_500_gb1dbs0_4_step2_80XRM',
 '190304_142547:andrea_crab_Zprime_FV_500_gb1dbs0_7_step2_80XRM',
 '190304_140921:andrea_crab_Zprime_FV_500_gb1dbs0_5_step2_80XRM',
 '190304_143157:andrea_crab_Zprime_FV_500_gb1dbs0_8_step2_80XRM',
 '190304_134020:andrea_crab_Zprime_FV_500_gb1dbs0_3_step2_80XRM',
 '190304_180546:andrea_crab_Zprime_FV_500_gb1dbs0_11_step2_80XRM',
 '190228_211034:algomez_crab_tthbb13_PostProcMEAnalysis_noME_SingleMuon_Run2018B_102X_v00',
 '190301_162902:tsusa_crab_forMarcoBeamSpotSel4_2017',
 '190228_211712:algomez_crab_tthbb13_PostProcMEAnalysis_noME_EGamma_Run2018B_102X_v00',
 '190302_013241:sciaba_crab_HC-102-T2_CH_CERN-72114-20190301032911',
 '190303_175822:tsusa_crab_forMarcoBeamSpotNewNoCW_2017',
 '190303_175611:tsusa_crab_forMarcoBeamSpotNew_2017',
 '190228_164125:algomez_crab_tthbb13_PostProcMEAnalysis_noME_ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8_102X_v00',
 '190305_181550:algomez_crab_tthbb13_PostProcMEAnalysis_withME_ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8_102X_v01',
 '190305_181439:algomez_crab_tthbb13_PostProcMEAnalysis_withME_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_102X_v01',
 '190228_160016:algomez_crab_tthbb13_PostProcMEAnalysis_noME_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_102X_v00',
 '190301_154858:leac_crab_MC_2017_ULgt_newPU_FEVT',
 '190228_211747:algomez_crab_tthbb13_PostProcMEAnalysis_noME_EGamma_Run2018C_102X_v00',
 '190228_111812:smoortga_crab_DIGIRAW_smeftttcc',
 '190304_135142:fernance_crab_DisplacedSUSY_squarkToQuarkChi_MSquark_1500_MChi_494_ctau_160mm_TuneCP5_14TeV_pythia8_DRPREMIX_step1',
 '190220_180659:mcitron_crab_SMS_gluinoGMSB_2017_grid_PUMix_gluinoGMSB_M1600_ctau100000p0',
 '190305_112843:vannerom_crab_ZprimeToTauprimeTauprime_MTau1_QTau1_DIGI_test',
 '190220_180216:mcitron_crab_SMS_gluinoGMSB_2017_grid_PUMix_gluinoGMSB_M1200_ctau5000p0',
 '190225_190655:qwang_crab_HIMB1_LmTree_v2',
 '190228_211207:algomez_crab_tthbb13_PostProcMEAnalysis_noME_SingleMuon_Run2018D_102X_v00',
 '190305_003651:jviinika_crab_TkAlJetHTReconstruction_Run2017ABCDEF-v1_10keventPerIov',
 '190304_191725:rpradhan_crab_PbPb2015_V0',
 '190228_182205:rpradhan_crab_PbPb2015_V0',
 '190227_164922:rpradhan_crab_PbPb2015_V0',
 '190227_211902:lguzzi_crab_WTau3Mu-RunIIFall18DRstep1_500k',
 '190226_175100:hatake_crab_PGun_step2_DIGI_10_2_11_E0_500_PU_v2',
 '190228_210904:algomez_crab_tthbb13_PostProcMEAnalysis_noME_SingleMuon_Run2018C_102X_v00',
 '190305_142449:meridian_crab_SingleGamma_Pt1To10_PU200',
 '190304_182812:ddesouza_crab_TT0_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190304_183433:ddesouza_crab_TT2_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190301_193954:ddesouza_crab_T01_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190303_050243:ddesouza_crab_T1X_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190301_193444:ddesouza_crab_T00_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190303_050355:ddesouza_crab_T2X_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190303_050840:ddesouza_crab_T3X_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190303_050041:ddesouza_crab_T0X_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190303_051915:ddesouza_crab_T4X_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190304_183311:ddesouza_crab_TT1_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190303_052244:ddesouza_crab_T6X_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190303_052029:ddesouza_crab_T5X_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190301_063526:jjay_crab_Ups1sMM_PbPb_MC_DIGIRAW_20190228',
 '190302_045940:rbi_crab_Pythia8_Ze10e10_m60120_pthat0_TuneCP5_5020GeV_GENSIM_1032_v1',
 '190305_082253:ikucher_crab_QCDPhoton_pThat-30_TuneCP5_PbPb_5p02TeV_pythia8_103X_upgrade2018_realistic_HI_v11_DIGI2RAW',
 '190304_191350:ikucher_crab_QCDPhoton_pThat-30_TuneCP5_PbPb_5p02TeV_pythia8_103X_upgrade2018_realistic_HI_v11_V2',
 '190302_172252:rbi_crab_Pythia8_Ze10e10_m60120_pthat0_TuneCP5_5020GeV_DIGIRAW_1032_v1',
 '190304_162348:ikucher_crab_QCDPhoton_pThat-30_TuneCP5_PbPb_5p02TeV_pythia8_103X_upgrade2018_realistic_HI_v11',
 '190305_061349:jjay_crab_Ups1sMM_PbPb_MC_DIGIRAW_20190304',
 '190301_153318:rbi_crab_Pythia8_WToENu_TuneCP5_5020GeV_DIGIRAW_1032_v1',
 '190304_195636:rbi_crab_Pythia8_Ze10e10_m60120_pthat0_TuneCP5_5020GeV_DIGIRAW_1032_v2',
 '190228_211857:algomez_crab_tthbb13_PostProcMEAnalysis_noME_EGamma_Run2018D_102X_v00',
 '190304_183042:ddesouza_crab_TT0_mc_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190304_183251:ddesouza_crab_TT1_mc_FemtopPb8TeV_1D_data_mix5_1_MBana',
 '190304_183500:ddesouza_crab_TT2_mc_FemtopPb8TeV_1D_data_mix5_1_MBana']

In [21]:
[x for x in prebadtasks if x in baddict['task']]

[]

In [22]:
prebadid = [13243219,
 13482678,
 13767499,
 13475561,
 13754501,
 13749780,
 13756230,
 13783383,
 13504221,
 13413676,
 12933850,
 13733264,
 13403730,
 13751582,
 13454467,
 13439723,
 13470231,
 13767559,
 13767403,
 13374215]

In [24]:
baddict['d_dataset_id']

[13740661,
 13442015,
 12887084,
 13739990,
 13477999,
 13740665,
 3854727,
 13504221,
 13740674,
 13740793,
 12682774,
 13740673,
 13740666,
 13703339]

In [26]:
[x for x in baddict['d_dataset_id'] if x in prebadid]

[13504221]

In [ ]:
goodfwjr = goodfwjr[~goodfwjr.task.isin(baddict['task'])]

In [ ]:
baderror = [x for x in goodads['ErrorType'][goodads.isboth == 0].unique().tolist() if x not in goodads['ErrorType'][goodads.isboth == 1].unique().tolist()]
guess = cleanads[cleanads.day == 1551484800.0][cleanads.ErrorType.isin(baderror)]
print(sum(guess.isboth))
print(len(guess))
print(baderror)
print(len(cleanads[cleanads.day == 1551484800.0][cleanads.isboth == 0]))
print(cleanads.ErrorType.unique())
print(sum(cleanads[cleanads.day == 1551484800.0][cleanads.ErrorType == 'Executable']['isboth']))
print(len(cleanads[cleanads.day == 1551484800.0][cleanads.ErrorType == 'Success']['isboth']))

In [ ]:
goodads.Status.unique()

In [ ]:
goodads[(goodads.ErrorType != 'Success') | (goodads.Status == 'Removed')].Status.unique()

In [ ]:
#these are the acceptions to the error rule.  Only success shown in fwjr. I dont expect to be able to explaine these
goodads[goodads.ErrorType != 'Success'][goodads.isboth == 1]

In [ ]:
for i in goodads.RequestCpus.unique().tolist():
    print(i)
    print(sum(goodads[goodads.RequestCpus == i].isboth)/len(goodads[goodads.RequestCpus == i].isboth))
    print(len(goodads[goodads.RequestCpus == i].isboth))

In [ ]:
print(np.sort(cleanads.CRAB_Retry.unique().tolist()))

In [ ]:
for i in np.sort(cleanads.CRAB_Retry.unique().tolist()):
    print(i)
    if len(goodads[goodads.CRAB_Retry == i].isboth) != 0:
        print(sum(goodads[goodads.CRAB_Retry == i].isboth)/len(goodads[goodads.CRAB_Retry == i].isboth))
    print(len(goodads[goodads.CRAB_Retry == i].isboth))

In [ ]:
print('Original percentage of jobs in both from classads on reasonable days perspctive:')
print(sum(goodads.isboth)/len(goodads.isboth))
print('Percentage of only 13 or less retries:')
print(sum(goodads[goodads.CRAB_Retry < 13.5].isboth)/len(goodads[goodads.CRAB_Retry < 13.5].isboth))
print('Percentage of 14 or more retries:')
print(sum(goodads[goodads.CRAB_Retry >13.5 ].isboth)/len(goodads[goodads.CRAB_Retry > 13.5].isboth))


In [ ]:
#shows how many unique values on good days of fwjr appear only in fwjr
mfwjr = goodfwjr#fwjr[fwjr.day.isin([1551484800, 1551571200, 1551657600])]
for i in mfwjr:
    print(i)
    if len(mfwjr[i][mfwjr.isboth == 0].unique())/len(mfwjr[i])>.01:
        print('')
        continue
    print(len([x for x in mfwjr[i][mfwjr.isboth == 0].unique().tolist() if x not in mfwjr[i][mfwjr.isboth == 1].unique().tolist()]))
    print(len(mfwjr[i][mfwjr.isboth == 0].unique().tolist()))
    print(len(mfwjr[i].unique().tolist()))

In [ ]:
i = 'd_dataset_id'
badtask = [x for x in mfwjr[i][mfwjr.isboth == 0].unique().tolist() if x not in mfwjr[i][mfwjr.isboth == 1].unique().tolist()]

In [ ]:
badtask

In [ ]:
goodfwjr = fwjr[fwjr.day.isin([1551484800, 1551571200, 1551657600, 1551744000])]#[fwjr.CRAB_Retry == '0']
len(goodfwjr)

In [ ]:
#my algorithm for fwjr

#creates an array for each field in the dataframe
#each value in the array represents a unique value in that field
#outputs the standard deviation then the mean of this array
#these numbers can be used to tell how useful each field should be for seperating based on isboth
#a useful field is one with a lower standard deviation and a higher mean
#Note: All values in the array are between 0 and 0.5 so the closer the second number is to 0.5,
#     the better that field should be as a predictor.

#iterates through fields in goodads
for field in goodfwjr:
    #creates and refreshes an array for each field
    prayer = []
    print(field)
    #skips to the next field if the current one has too many unique values for the code to run quickly
    #can be changed as desired
    if len(goodfwjr[field].unique())>5000:
        print(len(goodfwjr[field].unique()))
        continue
    #iterates through unique values of the field we are looking at.
    for i in goodfwjr[field].unique().tolist():
        #not sure why but sometimes there arent actually any jobs with a certain value that comes up
        #we have to skip these values to avoid division by zero in the next step
        if len(goodfwjr[goodfwjr[field] == i]['isboth']) == 0:
            continue
        #creates a value for the array equal to the absolute value of (0.5 - x)
        #where x is the precentage of jobs in both with with one particular value in the current fields
        newval = abs(0.5-sum(goodfwjr[goodfwjr[field] == i]['isboth'])/len(goodfwjr[goodfwjr[field] == i]['isboth'])) 
        prayer.append(newval)
    #print the field name, standard deviation of usefulness array,  
    #mean of usefulness array, and number of unique values in the field
    print(np.std(prayer))
    print(np.mean(prayer))
    print(len(goodfwjr[field].unique()))


In [ ]:
for i in np.sort(goodfwjr.CRAB_Retry.unique().tolist()):
    print(i)
    if len(goodfwjr[goodfwjr.CRAB_Retry == i].isboth) != 0:
        print(sum(goodfwjr[goodfwjr.CRAB_Retry == i].isboth)/len(goodfwjr[goodfwjr.CRAB_Retry == i].isboth))
    print(len(goodfwjr[goodfwjr.CRAB_Retry == i].isboth))

In [ ]:
for i in goodfwjr.host.unique().tolist():
    print(i)
    if len(goodfwjr[goodfwjr.host == i].isboth) != 0:
        print(sum(goodfwjr[goodfwjr.host == i].isboth)/len(goodfwjr[goodfwjr.host == i].isboth))
    print(len(goodfwjr[goodfwjr.host == i].isboth))

In [ ]:
#Creates an array of bad tasks to be removed from the set

badtask = []
#iterates through unique values of the desired field, in this case 'task'
for i in goodfwjr.task.unique().tolist():    
    #seperates good tasks from bad tasks by skipping any tasks for which more than 
    #half the jobs are in both
    if sum(goodfwjr[goodfwjr.task == i].isboth)/len(goodfwjr[goodfwjr.task == i].isboth)> 0.5:
        continue
    print(i)
    print(len(goodfwjr[goodfwjr.task == i].isboth))
    #adds the badtasks to the array
    badtask.append(i)


In [ ]:
#Creates an array of bad tasks to be removed from the set

badd_id = []
#iterates through unique values of the desired field, in this case 'task'
for i in goodfwjr.d_dataset_id.unique().tolist():    
    #seperates good tasks from bad tasks by skipping any tasks for which more than 
    #half the jobs are in both
    if sum(goodfwjr[goodfwjr.d_dataset_id == i].isboth)/len(goodfwjr[goodfwjr.d_dataset_id == i].isboth)> 0.5:
        continue
    print(i)
    print(len(goodfwjr[goodfwjr.d_dataset_id == i].isboth))
    #adds the badtasks to the array
    badd_id.append(i)


In [ ]:
#badtask = [x for x in mfwjr['task'][mfwjr.isboth == 0].unique().tolist() if x not in mfwjr['task'][mfwjr.isboth == 1].unique().tolist()]
guess = fwjr[fwjr.day == 1551484800][fwjr.task.isin(badtask)]
print(sum(guess.isboth))
print(len(guess))
print(guess.day.unique())
print(len(fwjr[fwjr.day == 1551744000][fwjr.isboth == 0]))
badtask

In [ ]:
#Compares the original set of jobs with the set that has tasks in badtask, 
#and the set made by removing the jobs with tasks in bad tasks.

#Each row shows the percentage of jobs in that set that is in both, and the number of jobs in the set 
#To create the new good set, use the following code
#newset = goodfwjr[~goodfwjr.task.isin(badtask)]

print('Original:')
print('Percent:\t'+str(sum(goodfwjr.isboth)/len(goodfwjr))+'\tLength:\t'+str(len(goodfwjr)))
print('FWJR with Badtasks:')
print('Percent:\t'+str(sum(goodfwjr[goodfwjr.task.isin(badtask)].isboth)/len(goodfwjr[goodfwjr.task.isin(badtask)]))+'\tLength:\t'+str(len(goodfwjr[goodfwjr.task.isin(badtask)])))
print('Original - Badtasts:')
print('Percent:\t'+str(sum(goodfwjr[~goodfwjr.task.isin(badtask)].isboth)/len(goodfwjr[~goodfwjr.task.isin(badtask)]))+'\tLength:\t'+str(len(goodfwjr[~goodfwjr.task.isin(badtask)])))


In [ ]:
print('Original:')
print('Percent:\t'+str(sum(goodfwjr.isboth)/len(goodfwjr))+'\tLength:\t'+str(len(goodfwjr)))
print('FWJR with Bad dataset id:')
print('Percent:\t'+str(sum(goodfwjr[goodfwjr.d_dataset_id.isin(badd_id)].isboth)/len(goodfwjr[goodfwjr.d_dataset_id.isin(badd_id)]))+'\tLength:\t'+str(len(goodfwjr[goodfwjr.d_dataset_id.isin(badd_id)])))
print('Original - Bad dataset id:')
print('Percent:\t'+str(sum(goodfwjr[~goodfwjr.d_dataset_id.isin(badd_id)].isboth)/len(goodfwjr[~goodfwjr.d_dataset_id.isin(badd_id)]))+'\tLength:\t'+str(len(goodfwjr[~goodfwjr.d_dataset_id.isin(badd_id)])))


In [ ]:
goodads.ErrorType.unique()
goodads = goodads[goodads.ErrorType == 'Success']

In [ ]:
badd_datasetid = [x for x in mfwjr['d_dataset_id'][mfwjr.isboth == 0].unique().tolist() if x not in mfwjr['d_dataset_id'][mfwjr.isboth == 1].unique().tolist()]
guess2 = fwjr[fwjr.d_dataset_id.isin(badtask)]
print(sum(guess2.isboth)

In [ ]:
cleanads = cleanads[cleanads.Status == 'Completed']
cleanads = cleanads[cleanads.ErrorType == 'Success']

In [ ]:
fwjr.day.unique()

In [ ]:
cleanads.day.unique()

In [ ]:
treetest={}
treetest['CRAB_Id'] = LabelEncoder().fit_transform(goodads['CRAB_Id'])
for p in cleanads:
    if type(goodads[p].iloc[1]) is not str:   
        treetest[p] = goodads[p]
        continue
    if (goodads[p].nunique()/len(goodads[p])>.5):
        continue
    if p == 'CMS_TaskType':
        continue
    else:
        treetest[p] = LabelEncoder().fit_transform(goodads[p])
treetest = pd.DataFrame(treetest)
#treetest = treetest[treetest.day.isin([1551484800.0, 1551571200.0, 1551657600.0, 1551744000.0])]
pd.DataFrame(treetest).head()

In [ ]:
sum(treetest.isboth)/len(treetest)

In [ ]:
for i in np.sort(fwjr.day.unique().tolist()):
    print(datetime.date.fromtimestamp(i))
#predictors.remove('CRAB_Workflow')
#treetest = treetest[treetest.ScheddName == 0.0]

In [ ]:
[*treetest]

In [ ]:
midtest = treetest
#midtest = treetest[treetest.day != 1551398400.0]
#midtest = midtest[midtest.day == 1551571200.0]

target = 'isboth'
predictors = ['CRAB_Id','RecordTime','day','d_dataset_id','CRAB_Workflow','CRAB_UserHN','RequestCpus','ScheddName','WallClockHr']
#predictors = []
#for i in [*treetest]:
 #   if i == target or i == 'day':
  #      continue
   # if type(treetest[i].loc[3]) is str:   
    #    continue
    #predictors.append(i)
#predictors.remove('ErrorType')
  
# defining predictors
X = midtest[predictors]

# definig the target variable and mapping it to 1 if in fwjr and 0 otherwise
y = midtest[target]

# instantiating the model
tree = DecisionTreeClassifier(min_samples_split = 4,max_depth = 10) 

# fitting the model
model = tree.fit(X,y)
treetest.head()

In [ ]:

dot_data = StringIO()  

export_graphviz(
    model, 
    out_file = dot_data,  
    filled=True, rounded=True, proportion=False,
    special_characters=True, 
    feature_names=X.columns,
    class_names=["Only Classads", "Both"]
)  

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

Image(graph.create_png())

In [ ]:
print(sum(treetest[treetest.ScheddName == 11].isboth)/len(treetest[treetest.ScheddName == 11].isboth))
print(len(treetest[treetest.ScheddName == 11].isboth))
print(len(treetest[treetest.isboth == 0]))

In [ ]:
for i in goodads.ScheddName.unique():
    print(i)
    print(sum(goodads[goodads.ScheddName == i].isboth)/len(goodads[goodads.ScheddName == i]))
    print(len(goodads[goodads.ScheddName == i]))

In [ ]:
print(np.mean(pd.to_numeric(goodads[goodads.isboth == 1].RecordTime))-np.mean(pd.to_numeric(goodads[goodads.isboth == 0].RecordTime)))
print(np.std(pd.to_numeric(goodads[goodads.isboth == 1].RecordTime)))
print(np.mean(pd.to_numeric(goodads[goodads.isboth == 0].RecordTime)))
print(np.std(pd.to_numeric(goodads[goodads.isboth == 0].RecordTime)))
print(np.mean(pd.to_numeric(goodads[goodads.isboth == 1].WallClockHr)))
print(np.std(pd.to_numeric(goodads[goodads.isboth == 1].WallClockHr)))
print(np.mean(pd.to_numeric(goodads[goodads.isboth == 0].WallClockHr)))
print(np.std(pd.to_numeric(goodads[goodads.isboth == 0].WallClockHr)))

In [ ]:
newx1 = treetest[treetest.day == 1551398400.0][predictors]
newy1 = treetest[treetest.day == 1551398400.0][target]
y1_test = model.predict(newx1)

#shifted later so that fwjr appears far more later in time
newx5 = treetest[treetest.day == 1551744000.0][predictors]
newy5 = treetest[treetest.day == 1551744000.0][target]
y5_test = model.predict(newx5)

In [ ]:
#True Negative     False Positive
#False Negative    True Positive
print(confusion_matrix(newy1,y1_test))

In [ ]:
print(len(newy1))
print(sum(newy1))
print(len(newy5))
print(sum(newy5))

In [ ]:
print(confusion_matrix(newy1,y1_test))

In [ ]:
print(confusion_matrix(newy5,y5_test))#10

In [ ]:
confusion_matrix(newy5,y5_test)[0,1]

In [ ]:
model.decision_path(newx5)

In [ ]:
sum(fwjr[fwjr.day.isin([1551657600])].isboth)/len(fwjr[fwjr.day.isin([1551657600])].isboth)

In [ ]:
temp = cleanads
cleanads = fwjr[fwjr.day.isin([1551484800, 1551571200, 1551657600, 1551744000])]

treetest={}
treetest['CRAB_Id'] = LabelEncoder().fit_transform(cleanads['CRAB_Id'])
for p in cleanads:
    if type(cleanads[p].loc[1]) is not str:   
        treetest[p] = cleanads[p]
        continue
    if (cleanads[p].nunique()/len(cleanads[p])>.8):
        continue
    if p == 'CRAB_Id':
        continue
    else:
        treetest[p] = LabelEncoder().fit_transform(cleanads[p])
cleanads = temp
pd.DataFrame(treetest).day.unique()


In [ ]:
treetest = pd.DataFrame(treetest)
midtest = treetest[treetest.day != 1551657600]
midtest = midtest[midtest.day != 1551571200]


target = 'isboth'
predictors = []
for i in [*treetest]:
    if i == target or i == 'day':
        continue
    if type(treetest[i].loc[1]) is str:   
        continue
    predictors.append(i)
    
# defining predictors
X = midtest[predictors]

# definig the target variable and mapping it to 1 if in fwjr and 0 otherwise
y = midtest[target]

# instantiating the model
tree = DecisionTreeClassifier(max_depth = 6, min_samples_leaf=90) 

# fitting the model
model = tree.fit(X,y)
print('done')

In [ ]:
dot_data = StringIO()  

export_graphviz(
    model, 
    out_file = dot_data,  
    filled=True, rounded=True, proportion=False,
    special_characters=True, 
    feature_names=X.columns,
    class_names=["Only Classads", "Both"]
)  

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  

Image(graph.create_png())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(X, y)

# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(4, 3))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
this will give me a warning

In [ ]:
# defining predictors
X = midtest[predictors]

# definig the target variable and mapping it to 1 if in fwjr and 0 otherwise
y = midtest[target]

# instantiating the model
false_positive = []
false_negative = []
true_positive = []
true_negative = []
for i in range(1,30):
    tree = DecisionTreeClassifier(max_depth = i, min_samples_leaf=90) 
    model = tree.fit(X,y)
    newx5 = treetest[treetest.day == 1551744000.0][predictors]
    newy5 = treetest[treetest.day == 1551744000.0][target]
    y5_test = model.predict(newx5)
    false_positive.append(confusion_matrix(newy5,y5_test)[0,1])
    false_negative.append(confusion_matrix(newy5,y5_test)[1,0])
    true_positive.append(confusion_matrix(newy5,y5_test)[1,1])
    true_negative.append(confusion_matrix(newy5,y5_test)[0,0])

In [ ]:
plt.plot(false_negative,'b')
plt.plot(false_positive,'y')
plt.plot(true_positive,'g')
plt.plot(true_negative,'r')
plt.show()

In [ ]:
plt.plot(false_negative)
plt.plot(false_positive)
plt.plot(true_positive,'g')
plt.plot(true_negative,'r')
plt.show()

In [ ]:
print(true_positive[45:55])
print(false_positive[45:55])
print(false_negative[45:55])

In [ ]:

print(true_positive[9])
print(true_negative[9])
print(false_positive[9])
print(false_negative[9])

In [ ]:
print(true_positive[0:10])
print(false_positive[0:10])
print(false_negative[0:10])

In [ ]:
true_positive[4]

In [ ]:
arrx = [1564617600,1564704000,1564790400,1564876800,1564963200,1565049600]
arry = []
for i in arrx:
    arry.append(len(fwjr[fwjr.day == i])-sum(fwjr[fwjr.day == i]['isclassads']))
plt.plot(arry)
plt.show()

In [ ]:
arry

In [ ]:
print(len(cleanads[cleanads.isboth == 1]))
print(len(cleanads))
print(len(cleanads[cleanads.isboth == 1])/len(cleanads))
print(len(fwjr))

In [ ]:
print(cleanads['day'].unique())

In [ ]:
#creates a dictionary to make each string into a number for the classifier
#each field will map one value to one number for example:
#{'T2_US_UCSD': 9} maps ucsd to 9 and is in the 'CMSSite' field

#the dictionary is used with the command df.replace(bigdict)
bigdict = {}
for i in cleanads:
    cleanads[i] = pd.to_numeric(cleanads[i],errors='ignore')
cleanads['CRAB_Id'] = pd.to_numeric(cleanads['CRAB_Id'],errors='coerce')

for p in cleanads:
    if type(cleanads[p].loc[1]) is not str:   
        continue
    if (len(cleansortedinfo[p])/len(cleanads[p])>.8):
        continue
    counter = 0
    littledict = {}
    for i in cleansortedinfo[p]:
        littledict.update({i:counter})
        counter+=1
    bigdict.update({p:littledict})

#dictionary can be updated using the following method:    
#bigdict.update({'CMSPrimaryPrimaryDataset':{'tester':0}})

In [ ]:
semisortedinfo = {}
res = defaultdict(list) 
for p in classads:
    res = defaultdict(list) 
    for key, val in sorted(classads[p].items()): 
        res[val].append(key)
    semisortedinfo.update({p:dict(res)})

In [ ]:
datetime.date.fromtimestamp(1564617600.0)

In [ ]:
treetest.to_pickle('treetest.pkl')
cleanads